In [351]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import operator
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import json
import ipywidgets as widgets
import functools
from IPython.display import display
from ipywidgets import Image
from IPython.display import display, Javascript
from IPython import display as display_p
import os
import elo

# Tournament 

## Input

As exposed in the cell below, the tournament takes a profile input, detailing what each profile is about.
 Here we use three different profiles for our tournament, the teacher profile, the student profile and the gender delegate profile.
 
## Execution
 The tournament matches each generated report for each profile against one another, and the user chooses the one they like the most. 
 
## Output

As outputs we get the tournament results and the inner representation of these results, which will then be inputed into the program for a visual representation of each one.
As the final output, we will have a score for each report presented to the user in the form of a result.json and 

In [318]:
# INPUT
student_profile = {
    "age": 18,
    "class": "student",
    "gender": "female",
    "nota": "7.5",
    "relevant_variables": ["grade", "course"]
    "comparison_groups": ["course", "all"]
    "description": "wants to know how its own grade (given) compares to the rest of the students in its course (given)",
    "curso": "2003-04"
}
teacher_profile = {
    "age": 48,
    "class": "teacher",
    "gender": "female",
    "relevant_variables": ["course", "grade"]
    "description": "wants to know the grade of their class (year here) in comparison to the rest of the years",
    "curso": "2003-04"
}
gender_delegate = {
    "age": 38,
    "class": "manager",
    "relevant_variables": ["gender", "grade"],
    "comparison_groups": ["male", "female"]
    "description": "wants to know how students of different gender have been performing over the years",
    "gender": "male"
}

SyntaxError: invalid syntax (<ipython-input-318-35564ce35eb7>, line 8)

In [ ]:
%matplotlib inline
df = pd.read_csv("./data/notas_merged.csv")

del df["NUMORD"]
del df["SUBTIPO"]
del df["CENTRO"]
del df["PLAN"]
del df["NOTA_BASE_4"]
categorical_columns = ["CURSO","ANOCON","MUNICIPIO","PROVINCIA","DESPLAN","GENERO"]

COLORS = ["red","blue","yellow","pink","purple","green"]
DESCRIPTIONS = {
    "CURSO": ["There are ", " students from the year "],
    "ANOCON": ["There are ", " students who graduated in "],
    "MUNICIPIO": ["There are ", " students from the municipality of"],
    "PROVINCIA": ["There are ", " students from the province of "],
    "DESPLAN": ["There are ", "students of the plan "],
    "GENERO": ["The are ", " students of gender "]
}

    
category_counts = {}
for c in categorical_columns:
    df[c] = df[c].astype("category")
    category_counts[c] = dict(df[c].value_counts() )

df = df.sort_values("CURSO")

In [344]:
def generate_category_count_pie(c,title=None):
    """Generates pie """
    p = {}
    p["title"] = title
    p["arg"] = [v for v in category_counts[c].values()]
    p["labels"] = [k for k in category_counts[c].keys()]
    p["colors"]=[random.choice(COLORS) for c in range(0,2)]
    return p

def generate_grade_plot(X, title=None):
    fig_info = {}
    fig_info["title"]=title
    fig_info["arg"] = X["NOTA_BASE_10"]
    fig_info["color"] = random.choice(COLORS)
    fig_info["legend"] = title
    return fig_info

def generate_student_report(profile, X):
    plots = {}
    c =  "CURSO"
    course = profile["curso"]
    
    X_course = X[X[c] == course]
    X_other = X[X[c] != course]

    dn1 = "Your grade is " + profile["nota"] + ". The median grade of your course is " + str(np.median(X_course["NOTA_BASE_10"])) + " The median grade for other courses is around" + str(np.median(X_other["NOTA_BASE_10"]))
    dn2 =  "Your grade is " + profile["nota"] + ". The median grade of your course is " + str(np.median(X_course["NOTA_BASE_10"]))
    dn3 = "Your grade is " + profile["nota"] 
    dn4 = "The median grade of your course is " + str(np.median(X_course["NOTA_BASE_10"])) + " The median grade for other courses is around" + str(np.median(X_other["NOTA_BASE_10"]))

    plots_gradecmp_courses= [generate_grade_plot(X_course,"Median ") for i in range(0,4)]
    plots_gradecmp_all = [generate_grade_plot(X_other,"Other courses performance") for i in range(0,4)]
    plots_dist_all = [generate_category_count_pie(c,"CURSO") for i in range(0,4)]
    
    # optionally 
    return [plots_gradecmp_courses, plots_gradecmp_all, plots_dist_all]
    
def generate_teacher_report(profile, X): 
    plots = {}
    c =  "CURSO"
    course = profile["curso"]
    
    X_course = X[X[c] == course]
    X_other = X[X[c] != course]
    
    dn1 = "The median grade of your course is " + str(np.median(X_course["NOTA_BASE_10"])) + " The median grade for other courses is around" + str(np.median(X_other["NOTA_BASE_10"]))
    dn2 =  "The median grade of your course is " + str(np.median(X_course["NOTA_BASE_10"]))
    dn4 = "The median grade of your course is " + str(np.median(X_course["NOTA_BASE_10"])) + " The median grade for other courses is around" + str(np.median(X_other["NOTA_BASE_10"]))

    plots_gradecmp_courses= [generate_grade_plot(X_course,"your course's grade distribution") for i in range(0,4) ]
    plots_gradecmp_all = [generate_grade_plot(X_other,"all courses grade distribution")  for i in range(0,4)]
    plots_dist_all = [generate_category_count_pie(c,"CURSO") for i in range(0,4)] 
    generate_category_count_pie
    # optionally 
    return [plots_gradecmp_courses, plots_gradecmp_all, plots_dist_all]

def generate_delegate_report(profile, X):
    plots = {}
    
    X_h = X[X["GENERO"] == "H"]
    X_m = X[X["GENERO"] != "H"]
    
    dn1 = "The median grade of the female students is" + str(np.median(X_m["NOTA_BASE_10"])) + ".\n The median grade for male students is " + str(np.median(X_h["NOTA_BASE_10"])) + "."
    dn2 =  "The median grade of the female students is" + str(np.median(X_m["NOTA_BASE_10"]))
    dn3 = " The median grade for male students is " + str(np.median(X_h["NOTA_BASE_10"]))
    
    plots_gradecmp_female= [generate_grade_plot(X_m, "female") for i in range(0,4)]
    plots_gradecmp_male = [generate_grade_plot(X_h, "male") for i in range (0,4)] 
    plots_dist_all = [generate_category_count_pie("GENERO", "Gender distribution") for i in range(0,4 )]
    
    # optionally 
    return [plots_gradecmp_female, plots_gradecmp_male, plots_dist_all]

def givemefigures(hist , pie, mode):
    #fig = plt.figure()
    #plt.title(hist.title + pie.title)
    if mode == "histpie":
        fig, (ax1, ax2) = plt.subplots(1, 2)
        plt.title(hist["title"] + " " + pie["title"])
        ax1.hist(hist["arg"],color=hist["color"])
        red_patch = mpatches.Patch(color=hist["color"], label=hist["legend"])
        ax1.legend(handles=[red_patch])
        ax2.pie(pie["arg"],labels=pie["labels"],autopct=None,colors=pie["colors"])
    elif mode == "histhist":
        fig, (ax1, ax2) = plt.subplots(1, 2)
        plt.title(hist["title"]+ " " + pie["title"])
        ax1.hist(hist["arg"],color=hist["color"])
        red_patch = mpatches.Patch(color=hist["color"], label=hist["legend"])
        ax1.legend(handles=[red_patch])
        ax2.hist(pie["arg"],color=pie["color"])
        blue_patch = mpatches.Patch(color=pie["color"], label=pie["legend"])
        ax2.legend(handles=[blue_patch])

    return fig

def create_reports(repo_someone):
    #pair each with one of the others
    chosen_ones = []
    for j in range(0,len(repo_someone[1])):
        chosen_ones.append((repo_someone[1][j], random.choice(repo_someone[2]),"histpie"))
    for i in range(0,len(repo_someone[0])):
        chosen_ones.append((repo_someone[0][i], random.choice(repo_someone[2]),"histpie"))
        chosen_ones.append((repo_someone[0][i], random.choice(repo_someone[1]),"histhist"))


        
    return chosen_ones

def write_png(tournament_name, reports):
    figures = [] 
    
    if not os.path.exists(tournament_name):
        os.mkdir(tournament_name)
        print("Directory " , tournament_name ,  " Created ")
    else:    
        print("Directory " , tournament_name ,  " already exists")

    for i in range(0,len(reports)):
        figures.append(givemefigures(reports[i][0],reports[i][1],reports[i][2]))

    i=0;
    for fig in figures:
        fig.savefig(tournament_name + "/" + str(i)+".png")
        plt.close(fig)
        i+=1
    return reports

In [354]:
#dictionary with keys range 0 to 9 and values all numbers from 0 to nine removin i
def tournament(tournament_name):
    if tournament_name == "student" : 
        student_reports = create_reports(generate_student_report(student_profile, df))
        repo_student = write_png(tournament_name , student_reports)
    elif tournament_name == "teacher":
        teacher_reports = create_reports(generate_teacher_report(teacher_profile, df))
        repo_teacher = write_png(tournament_name, teacher_reports)
    else:
        delegate_reports = create_reports(generate_delegate_report(gender_delegate, df))                              
        repo_delegate = write_png(tournament_name, delegate_reports)
        
    tourney_scorer = dict([(i,list(range(0,i))) for i in range(1,9)])
    displayed = {}
    results = {}
    def looparound(next_index):
        global results
        global displayed
        
        def pick_one(i):
            global results
            global displayed
            def other(k):
                return 1 if k == 2 else 2
            "Button(description='Choice 0', style=ButtonStyle())"
            
            winner = i.description.split(" ")[-1]
            loser = other(i)
            new_elos = elo.rate_1vs1(results[str(displayed[str(i)])],results[str(displayed[str(other(i))])])

            results[str(displayed[str(i)])] = new_elos[0]
            results[str(displayed[str(other(i))])] = new_elos[1]

            with open(tournament_name + '/result.json', 'w') as fp:
                json.dump(results, fp)
            looparound(next_index)
    
        def load_2(next_index):
            try:
                if len(tourney_scorer[next_index]) == 0:
                    next_index += 1
                    return load_2(next_index)
                else: 
                    first = tourney_scorer[next_index][0]
                    second = next_index
                    tourney_scorer[next_index].remove(first)
                    return [str(first),str(second)]      
            except Exception as e :
                print("Tournament finished!")
                with open(tournament_name + '/result.json', 'w') as fp:
                    json.dump(results, fp)

                return ["-1","-1"]

        display_p.clear_output(wait=True)
        exists = os.path.isfile(tournament_name + '/result.json')
        if exists:
            with open(tournament_name + '/result.json') as f:
                results = json.load(f)
            results = dict(results)
        else:
            results = dict((str(i),1000) for i in range(0,9))
        [first, second] = load_2(next_index)
        if ["-1","-1"] == [first,second]:
            if tournament_name == "student" : 
                return student_reports
            elif tournament_name == "teacher":
                return teacher_reports
            else:
                return delegate_reports 
                                         
        displayed = {
            "1": str(first),
            "2": str(second)
        }
        #actually display
        COLS = 2
        ROWS = 1
        IMAGES = [open(tournament_name + "/" + str(i)+".png", 'rb').read() for i in [first, second]]
        IMG_WIDTH = 450
        IMG_HEIGHT = 600

        rows = []

        for row in range(ROWS):
            cols = []
            for col in range(COLS):
                index = row * COLS + col
                image = widgets.Image(
                    value=IMAGES[index], width=IMG_WIDTH, height=IMG_HEIGHT
                )
                button = widgets.Button(description='Choice %d' % (index+1))
                # Bind the click event to the on_click function, with our index as argument
                button.on_click(functools.partial(pick_one))

                # Create a vertical layout box, image above the button
                box = widgets.VBox([image, button])
                cols.append(box)

            # Create a horizontal layout box, grouping all the columns together
            rows.append(widgets.HBox(cols))

    # Create a vertical layout box, grouping all the rows together
        result_disp = widgets.VBox(rows)
        display(result_disp)
    next_index = 1
    looparound(next_index)

In [334]:
Javascript("Jupyter.notebook.execute_cells([5])")
reports = tournament("student")

In [336]:
Javascript("Jupyter.notebook.execute_cells([4])")
tournament("teacher")

In [346]:
Javascript("Jupyter.notebook.execute_cells([4])")
tournament("gender_delegate")

In [350]:
elo.rate_1vs1(1000,1000)

(1005.0, 995.0)